In [1]:
import gitlab, yaml

with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

token = cfg['gitlab_token']

gl = gitlab.Gitlab('https://gitlab.com', private_token=token, api_version=4)
gl.auth()
project = gl.projects.get('simm/covid19/secir')
# commits = project.commits.list(all=True)

In [3]:
import pandas as pd
from io import StringIO
from tqdm.auto import tqdm
tqdm.pandas()

# to retrieve all commits, clone the repo and run the following command in terminal:
#  git rev-list HEAD img/dynamic/Rt_rawData/Germany_Rt.csv >> all-commits.txt
# then copy the file with the results into this folder here

all_commits = pd.read_csv('../data-raw/Braunschweig/all-commits.txt', sep = '\n', header=None)

In [5]:
files_not_found = 0
for c in tqdm(all_commits.iloc[:,0]):
    # download file for a specific commit
    try:
        file = project.files.raw(file_path='img/dynamic/Rt_rawData/Germany_Rt.csv', ref=c)
    except:
        files_not_found += 1
    # convert file from byte to str
    s = str(file, 'utf-8')
    # read file as DataFrame
    df = pd.read_csv(StringIO(s))
    df = df.dropna()
    # write DataFrame to .csv-file
    date = df['date'][len(df) - 1]
    df.to_csv("../data-raw/Braunschweig/" + date + "-Braunschweig_raw.csv", index_label=False, index=False)

100%|██████████| 280/280 [02:23<00:00,  1.96it/s]


In [2]:
"""
previous try to retrieve the commits via the API: does not work properly, only 26 commits for this file!

from collections import defaultdict
from tqdm.auto import tqdm
tqdm.pandas()
# file paths and a list of commits which create/modify/delete the file
file_map = defaultdict(list)

# save file names if their name changed
file_names = set()

for c in tqdm(commits):
    diff = c.diff()
    files_changed = set()
    for change in diff:
        files_changed.add(change['old_path'])
        files_changed.add(change['new_path'])
        # check if file name changed:
        if change['old_path'] != change['new_path']:
            file_names.add((change['old_path'],change['new_path'] ))
    for path in files_changed:
        file_map[path].append(c)

#  show list of commits which modified e.g. README.md
# print(file_map['README.md'])
"""

100%|██████████| 345/345 [03:09<00:00,  1.82it/s]


In [ ]:
# file_names # check this list for your specific file to see if it had a different name previously

In [13]:
"""
# save files
import pandas as pd
from io import StringIO

# for all commits which changed your specific file:
for c in tqdm(file_map['img/dynamic/Rt_rawData/Germany_Rt.csv']):
    # download file for a specific commit
    file = project.files.raw(file_path='img/dynamic/Rt_rawData/Germany_Rt.csv', ref=c.get_id())
    # convert file from byte to str
    s = str(file, 'utf-8')
    # read file as DataFrame
    df = pd.read_csv(StringIO(s))
    # write DataFrame to .csv-file
    df.to_csv("../data-raw/Braunschweig/" + c._attrs['created_at'][0:10] + "-Braunschweig_raw.csv", index_label=False, index=False)
"""

100%|██████████| 26/26 [00:29<00:00,  1.12s/it]
